In [1]:
from lxml import etree

In [2]:
from typing import List, Tuple

In [3]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = str(column.text).strip()
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [4]:
texts, labels = load_sentirueval_2016('bank_train_2016.xml')

In [5]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 9392, number of labels is 9392.


In [6]:
import random

In [7]:
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

neutral => @sberbank Слышала, что якобы если снимаешь с денежного вклада  более 50000 руб., то с суммы, превышающей 50000 руб., нужно платить налог?
negative => RT @findozor: СМИ: Россельхозбанк и Газпромбанк просят помощи у государства #банк #финансы http://t.co/Edw0eGcYih
positive => Порадовал @AlfabankBY отменой комиссии за пользование кредитом в этом месяце. Может совсем отменят.
neutral => RT @CP3Plan: райффайзен кредитная карта http://t.co/Pu680vSQvZ
neutral => @sberbank в основном ошибка «счет заблокирован», хотя все остальные переводы от физлица работают и у нас со счетом все нормально
negative => Сбербанк приостановит 19 января ряд онлайн-операций из-за техработ http://t.co/k3p4GeS1zA
neutral => ипотека втб 24 ставки http://t.co/n3cuX6J9gT
negative => RT @decijoqogure: Хуже кладбища может быть только Сбербанк.
neutral => Как будут работать офисы и удаленные системы обслуживания Сбербанка в новогодние праздники http://t.co/YSbtVoqAIt
neutral => http://t.co/MyuFBbqXQ8 Как взять 

In [8]:
positive_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'positive']
negative_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'negative']

In [9]:
for cur in positive_tweets[:5]: print(cur)

@sawik_shuster @YevhenS Главное чтоб банки СБЕР и ВТБ!!!
Самый выгодный автокредит в втб 24 http://t.co/CDJ5P40x7g
http://t.co/h6r6GdBe4H Легко можно получить денежный кредит ы втб 24 банке
Снижение процентной ставки по кредиту на недвижимость сбербанк http://t.co/MredO898wK
В наш сбербанк пришел американец и попросил сфоткаться с ним у банкомата ;D 
я ваще была в шоке и на фотке полюбому получилась смешная xD


In [10]:
for cur in negative_tweets[:5]: print(cur)

@ShtirliZ_ @Zhukova_olga @winzard @tereshenkov обязательно про сбербанк напишите! Временами похлеще интернет магазинов финты выкручивает)
@Umka_75 @Alfarius2012 @igor_sechin втб и сбер точно. вопрос-блокировка операций или запрет на кредиты?
Канада ввела санкции против Газпромбанка, ВЭБа, Новатэка и российских оборонных компаний
RT @letokot: Если Сбер и ВТБ попали под санкции, будет жопа
RT @360_tv_ru: #Канада ввела санкции против 10 оборонных и сырьевых компаний и банков #РФ. Среди них, в частности, "#Газпромбанк", "#Внешэк…


In [11]:
from nltk import word_tokenize

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize)

In [14]:
vectorizer.fit(texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x7fd8c66eb950>,
                vocabulary=None)

In [15]:
print(vectorizer.get_feature_names()[0:20])

['!', '#', '$', '%', '&', "''", '(', ')', '*^____^*', '*бесится*', '*трипл-смайлик-крик-мунка*', '+', '+3,49', '+300', '+5тыс', '+74955102933', '+в', ',', '-', '--']


In [16]:
print(len(vectorizer.get_feature_names()))

20240


In [17]:
X = vectorizer.transform(texts)

In [18]:
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [19]:
print(texts[0])

http://t.co/YEVHuvVGA1 Взять кредит тюмень альфа банк


In [20]:
print(X[0])

  (0, 6321)	1
  (0, 7199)	1
  (0, 7775)	1
  (0, 9093)	1
  (0, 9295)	1
  (0, 9866)	1
  (0, 12802)	1
  (0, 19056)	1


In [21]:
print(vectorizer.get_feature_names()[6321])

//t.co/yevhuvvga1


In [22]:
print(vectorizer.get_feature_names()[9866])

взять


In [23]:
print(vectorizer.get_feature_names()[19056])

тюмень


In [24]:
from sklearn.feature_extraction.text import TfidfTransformer

In [25]:
transformer = TfidfTransformer().fit(X)

In [26]:
X_transformed = transformer.transform(X)

In [27]:
print(X_transformed[0])

  (0, 19056)	0.5196718117883131
  (0, 12802)	0.1747696254947399
  (0, 9866)	0.2815105414658593
  (0, 9295)	0.20532647450968283
  (0, 9093)	0.26133302396332425
  (0, 7775)	0.10105244280985996
  (0, 7199)	0.09241698661915439
  (0, 6321)	0.7006726731639271


In [28]:
print(vectorizer.get_feature_names()[19056])

тюмень


In [29]:
print(vectorizer.get_feature_names()[7199])

:


In [30]:
tokens_with_IDF = list(zip(vectorizer.get_feature_names(), transformer.idf_))

In [31]:
for feature, idf in tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
3.144655 => #
6.276519 => $
5.636861 => %
5.647910 => &
4.134005 => ''
4.805386 => (
4.443938 => )
9.454573 => *^____^*
9.454573 => *бесится*
9.454573 => *трипл-смайлик-крик-мунка*
9.049108 => +
9.454573 => +3,49
9.454573 => +300
9.454573 => +5тыс
9.454573 => +74955102933
9.454573 => +в
2.753227 => ,
4.420872 => -
8.761426 => --


In [32]:
sorted_tokens_with_IDF = sorted(tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [33]:
for feature, idf in sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => *^____^*
9.454573 => *бесится*
9.454573 => *трипл-смайлик-крик-мунка*
9.454573 => +3,49
9.454573 => +300
9.454573 => +5тыс
9.454573 => +74955102933
9.454573 => +в
9.454573 => -4.8
9.454573 => -5
9.454573 => -6.4
9.454573 => -а
9.454573 => -круто
9.454573 => -осуществление
9.454573 => -подключение
9.454573 => -привлечение
9.454573 => -разменяйте
9.454573 => -россельхозбанка
9.454573 => -сами
9.454573 => -сервис


In [34]:
from sklearn.feature_selection import SelectPercentile, chi2

In [35]:
selector = SelectPercentile(chi2, percentile=20)

In [36]:
selector.fit(X_transformed, labels)

SelectPercentile(percentile=20, score_func=<function chi2 at 0x7fd8c1eaa1e0>)

In [37]:
selected_tokens_with_IDF = [tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]

In [38]:
print(len(selected_tokens_with_IDF))

4048


In [39]:
for feature, idf in selected_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
3.144655 => #
5.636861 => %
4.134005 => ''
4.805386 => (
4.443938 => )
9.454573 => *^____^*
9.049108 => +
9.454573 => +5тыс
2.753227 => ,
4.420872 => -
9.454573 => -россельхозбанка
3.175114 => .
7.012226 => ..
3.784692 => ...
9.454573 => ._
6.070183 => /
9.454573 => //t.co/01ftjc0bur
9.454573 => //t.co/0gfmj3s0gi
9.454573 => //t.co/0i4pf81ksd


In [40]:
selected_and_sorted_tokens_with_IDF = sorted(selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [41]:
for feature, idf in selected_and_sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => *^____^*
9.454573 => +5тыс
9.454573 => -россельхозбанка
9.454573 => ._
9.454573 => //t.co/01ftjc0bur
9.454573 => //t.co/0gfmj3s0gi
9.454573 => //t.co/0i4pf81ksd
9.454573 => //t.co/0jzv0jq959
9.454573 => //t.co/0lt5jdmk9l
9.454573 => //t.co/0s03a0v7po
9.454573 => //t.co/0ttwpnjnef
9.454573 => //t.co/0vpcvumoe8
9.454573 => //t.co/0wziwezr91
9.454573 => //t.co/0y05qjok2q
9.454573 => //t.co/1aa3jpbxqh
9.454573 => //t.co/1cjshtzrda
9.454573 => //t.co/1d3bb71qbt
9.454573 => //t.co/1eodyxqq0a
9.454573 => //t.co/1fxzerzsps
9.454573 => //t.co/1kiwdyic6t
